In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html

In [38]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [39]:
url = 'https://torgi.gov.ru/new/api/public/lotcards/rss?subjRF=50&chars=&lotStatus=SUCCEED&biddEndFrom=&biddEndTo=&pubFrom=&pubTo=&aucStartFrom=&aucStartTo=&catCode=4&text='
req = requests.get(url, verify=False)

soup = BeautifulSoup(req.text, features='xml')

In [40]:
num = soup.find_all('item')
description_list = []
for index in num:
    col = index.find(name='description')
    sel = html.unescape(col.text)
    description_list.append(sel)
print(description_list)

['<b>Параметры поиска:</b><br><b>Субъект РФ:</b> Московская область<br><b>Статус лота:</b> Состоялся<br><b>Категория имущества:</b> Земли сельскохозяйственного назначения<br><br><b>Список лотов:</b><br><b>Номер лота:</b> <a href="https://torgi.gov.ru/new/public/lots/lot/21000004710000001968_1">№ 21000004710000001968 лот № 1</a><br><b>Наименование лота (предмет торгов):</b> Аренда земельного участка 110 165 кв.м под пчеловодство в г.о. Егорьевск (КН 50:30:0060301:565)<br><b>Вид торгов:</b> Аренда и продажа земельных участков<br><b>Форма проведения торгов:</b> Электронный аукцион<br><b>Статус лота:</b> Состоялся<br><b>Электронная площадка:</b> https://i.rts-tender.ru/main/auction/Trade/Privatization/View.aspx?Id=91161&Guid=05fbaaf3-72ac-4e67-a7e1-390e68fb35d0#80a02c00-8969-4bf2-b498-93cb87eb5a59<br><b>Дата публикации:</b> 2022-06-23T13:57:34.656Z<br><b>Дата изменения:</b> 2022-08-09T09:36:02.866Z<br><br><b>Начальная цена:</b> <br>3987.0<br><br><b>Номер извещения:</b> <br><a href="https:/

In [41]:
links_all = []
for i in description_list:
    b = BeautifulSoup(i, features='html')
    link = b.find('a')
    link_final = link.get('href')
    links_all.append(link_final)

In [42]:
links_all

['https://torgi.gov.ru/new/public/lots/lot/21000004710000001968_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000000896_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001869_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001794_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001723_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001670_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001641_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001621_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001478_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001465_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001373_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001327_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001298_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000004710000001243_1',
 'https://torgi.gov.ru/new/public/lots/lot/21000

In [48]:
from selenium import webdriver
import time
driver = webdriver.Chrome('C:\\Users\\User\\Desktop\\chromedriver.exe')

C:\Users\User\AppData\Local\Temp\ipykernel_5600\4243471799.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\User\\Desktop\\chromedriver.exe')


In [ ]:
columns = ['Название лота в шапке','Вид аукциона','Номер лота','Начальная цена','Шаг аукциона','Вид торгов','Электронная площадка','Субъект местонахождения имущества','Дата и время начала подачи заявок','Дата и время окончания подачи заявок','Начало проведения торгов','Размер задатка','Категория объекта','Форма собственности','Срок аренды','Вид разрешённого использования земельного участка','Кадастровый номер земельного участка']
data = pd.DataFrame(columns=columns, index=[0,1,2,3,4,5,6,7,8,9,10])
data

In [49]:
def data_parse(link):
    driver.get(link)
    time.sleep(2)
    main_page = driver.page_source
    soup = BeautifulSoup(main_page, features='xml')
    
    # Ищу название лота в шапке
    head = soup.find_all(attrs={"class":{"header_title"}})
    header = head[0].text
    
    # Вид аукциона 
    auc = soup.find_all(attrs={"class":{"lotBiddForm with-small-bottom-gap"}}) 
    auction = auc[0].text
    
    # Номер лота
    links = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    link = links[0].text
    
    # Начальная цена
    start = soup.find_all(attrs={"class":{"prices__row__price-cell lotPrice"}}) 
    start_price = start[0].text
    
    # Шаг аукциона
    step_auction = start[1].text
    
    # Вид торгов
    auction_type = links[1].text
    
    # Электронная площадка
    rts_links = soup.find_all('a')
    rts_link = rts_links[-6].get('href')
    
    # Субъект местонахождения имущества
    place = links[3].text.rstrip()
    
    # Дата и время начала подачи заявок
    datetime_start = links[4].text
    
    # Дата и время окончания подачи заявок
    datetime_end = links[5].text
    
    # Начало проведения торгов
    datetime_start_auction = links[6].text
    
    # Размер задатка
    deposit_amount = soup.find_all(attrs={"class":{"attr_value"}})
    deposit = deposit_amount[5].text
    
    # Категория объекта
    object_type = deposit_amount[19].text
    
    # Форма собственности
    form = deposit_amount[20].text
    
    # Срок аренды
    rent_time = deposit_amount[23].text
    
    # Вид разрешённого использования земельного участка
    vid_done = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})
    vid = vid_done[1].text
    
    # Кадастровый номер земельного участка
    number_kad = vid_done[2].text
    
    ###############################################
    list_of_data = [header,auction,link,start_price,step_auction,auction_type,rts_link,place,datetime_start,datetime_end,datetime_start_auction,deposit,object_type,form,rent_time,vid,number_kad]
    return list_of_data

In [57]:
data_parse(links_all[0])

['Аренда земельного участка 110 165 кв.м под пчеловодство в г.о. Егорьевск (КН 50:30:0060301:565)',
 ' Электронный аукцион ',
 '№21000004710000001968, лот №1',
 '3987,00₽',
 '119,00 ₽',
 ' Аренда и продажа земельных участков ',
 'https://i.rts-tender.ru/main/auction/Trade/Privatization/View.aspx?Id=91161&Guid=05fbaaf3-72ac-4e67-a7e1-390e68fb35d0#80a02c00-8969-4bf2-b498-93cb87eb5a59',
 ' Московская область',
 ' 24.06.2022 09:00 (МСК)',
 ' 04.08.2022 18:00 (МСК)',
 ' 08.08.2022 12:00 (МСК)',
 '3987,00₽',
 'Земли сельскохозяйственного назначения ',
 'Иная собственность',
 '15 лет  ',
 ' Пчеловодство ',
 ' 50:30:0060301:565 ']

In [59]:
for column in data.columns:
    print(column)

Название лота в шапке
Вид аукциона
Номер лота
Начальная цена
Шаг аукциона
Вид торгов
Электронная площадка
Субъект местонахождения имущества
Дата и время начала подачи заявок
Дата и время окончания подачи заявок
Начало проведения торгов
Размер задатка
Категория объекта
Форма собственности
Срок аренды
Вид разрешённого использования земельного участка
Кадастровый номер земельного участка


# Достаю данные с конкретного лота

### Название лота в шапке

In [290]:
header = soup.find_all(attrs={"class":{"header_title"}})
header[0].text

'Аренда земельного участка 110 165 кв.м под пчеловодство в г.о. Егорьевск (КН 50:30:0060301:565)'

### Вид аукциона

In [291]:
auction = soup.find_all(attrs={"class":{"lotBiddForm with-small-bottom-gap"}}) 
auction[0].text

' Электронный аукцион '

### Номер лота

In [292]:
links = soup.find_all(attrs={"class":{"lotAttributeValue"}})

In [294]:
links[0].text

'№21000004710000001968, лот №1'

### Начальная цена 

In [297]:
start_price = soup.find_all(attrs={"class":{"prices__row__price-cell lotPrice"}}) 
start_price[0].text

'3987,00₽'

### Шаг аукциона

In [300]:
start_price[1].text

'119,00 ₽'

### Вид торгов

In [303]:
links[1].text

' Аренда и продажа земельных участков '

### Электронная площадка

In [352]:
rts_link = soup.find_all('a')
rts_link[-6].get('href')

'https://i.rts-tender.ru/main/auction/Trade/Privatization/View.aspx?Id=91161&Guid=05fbaaf3-72ac-4e67-a7e1-390e68fb35d0#80a02c00-8969-4bf2-b498-93cb87eb5a59'

### Субъект местонахождения имущества

In [364]:
place = links[3].text.rstrip()  
place

' Московская область'

### Дата и время начала подачи заявок

In [365]:
datetime_start = links[4].text
datetime_start

' 24.06.2022 09:00 (МСК)'

### Дата и время окончания подачи заявок

In [366]:
datetime_end = links[5].text
datetime_end

' 04.08.2022 18:00 (МСК)'

### Начало проведения торгов

In [367]:
datetime_start_auction = links[6].text
datetime_start_auction

' 08.08.2022 12:00 (МСК)'

### Размер задатка

In [371]:
deposit_amount = soup.find_all(attrs={"class":{"attr_value"}})
deposit_amount[5].text

'3987,00₽'

### Категория объекта

In [374]:
deposit_amount[19].text

'Земли сельскохозяйственного назначения '

### Форма собственности

In [377]:
deposit_amount[20].text

'Иная собственность'

### Срок аренды

In [380]:
deposit_amount[23].text

'15 лет  '

### Вид разрешённого использования земельного участка

In [384]:
vid = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})
vid[1].text

' Пчеловодство '

### Кадастровый номер земельного участка

In [386]:
vid[2].text

' 50:30:0060301:565 '

In [387]:
columns = ['Название лота в шапке','Вид аукциона','Номер лота','Начальная цена','Шаг аукциона','Вид торгов','Электронная площадка','Субъект местонахождения имущества','Дата и время начала подачи заявок','Дата и время окончания подачи заявок','Начало проведения торгов','Размер задатка','Категория объекта','Форма собственности','Срок аренды','Вид разрешённого использования земельного участка','Кадастровый номер земельного участка']

In [390]:
values = [header[0].text, auction[0].text, links[0].text]

In [405]:
data = pd.DataFrame(columns=columns, index=[0,1,2,3,4,5,6,7,8,9,10])

In [407]:
data.loc[0,'Название лота в шапке'] = header[0].text


In [408]:
data

,Название лота в шапке,Вид аукциона,Номер лота,Начальная цена,Шаг аукциона,Вид торгов,Электронная площадка,Субъект местонахождения имущества,Дата и время начала подачи заявок,Дата и время окончания подачи заявок,Начало проведения торгов,Размер задатка,Категория объекта,Форма собственности,Срок аренды,Вид разрешённого использования земельного участка,Кадастровый номер земельного участка
0,Аренда земельного участка 110 165 кв.м под пче...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
